In [26]:
import pickle
from pathlib import Path
from collections import Counter, defaultdict

In [30]:
def deserializeFileToObject(filename: str = 'objs.pkl') -> list:
    text_file = Path('../../etc/out') / filename
    with text_file.open('rb') as f:
        objects = pickle.load(f)
    return objects

def writeOutputsForDatasets(dataset_tables: dict[list], text_filename: str = 'project.out', latex_filename: str = 'project_latex.out', terminal_output: bool = True):
    text_file = Path('../../etc/out') / text_filename
    latex_file = Path('../../etc/out') / latex_filename
    with text_file.open('w') as f1, latex_file.open('w') as f2:
        for dataset, tables in dataset_tables.items():
            table1, table2, table_times = tables[0], tables[1], tables[2]
            # Write to terminal out
            if terminal_output:
                print(f'\nX{dataset:-^40}X\n')
                print(table1)
                print("\n")
                print(table2)
                print("\n")
                print(table_times)

            # Write to text file
            f1.write(f'\n\nX{dataset:-^50}X\n\n')
            f1.write(table1.to_string())
            f1.write("\n\n")
            f1.write(table2.to_string())
            f1.write("\n\n")
            f1.write(table_times.to_string())

            # Write to latex file
            f2.write(f'\n\nX{dataset:-^50}X\n\n')
            f2.write(table1.to_latex(float_format="%.2f"))
            f2.write("\n\n")
            f2.write(table2.to_latex())
            f2.write("\n\n")
            f2.write(table_times.to_latex())

In [31]:
dataset_tables = deserializeFileToObject()

In [32]:
def addTaxes(dataset_tables):
    for dataset, tables in dataset_tables.items():
        table1, table2, table3 = tables
        table1.loc['samp tax1'] = table1.loc['all'] - table1.loc['sway1']
        table1.loc['xpln tax1'] = table1.loc['sway1'] - table1.loc['xpln1']
        table1.loc['samp tax2'] = table1.loc['all'] - table1.loc['sway2']
        table1.loc['xpln tax2'] = table1.loc['sway2'] - table1.loc['xpln2']
        table1.loc['samp tax3'] = table1.loc['all'] - table1.loc['sway3']
        table1.loc['xpln tax3'] = table1.loc['sway3'] - table1.loc['xpln3']
    
    return dataset_tables

dataset_tables_with_taxes = addTaxes(dataset_tables)
writeOutputsForDatasets(dataset_tables_with_taxes)


X----------------SSM.csv-----------------X

           NUMBERITERATIONS-  TIMETOSOLUTION-
title                                        
all                     7.00           134.89
sway1                   5.10           120.04
sway2                   5.40           120.22
sway3                   5.15           104.69
top                     4.00            59.97
xpln1                   5.45           125.68
xpln2                   5.60           120.10
xpln3                   5.65           103.89
samp tax1               1.90            14.85
xpln tax1              -0.35            -5.64
samp tax2               1.60            14.67
xpln tax2              -0.20             0.12
samp tax3               1.85            30.20
xpln tax3              -0.50             0.80


               NUMBERITERATIONS- TIMETOSOLUTION-
all to all                     =               =
all to sway1                   ≠               =
all to sway2                   ≠               ≠
all to sway3         

In [27]:
def getDatasetScores(dataset_tables):
    scores = defaultdict(dict)
    for dataset, tables in dataset_tables.items():
        table1, table2, table3 = tables
        columns = list(table2.columns)
        equal2, better2, worse2 = 0,0,0
        equal3, better3, worse3 = 0,0,0
        for col in columns:
            multiplier = 1 if col[-1]=='+' else -1
            # for sway2
            if table2.loc['sway1 to sway2',col] == '=':
                equal2+=1
            else:
                sway1_val = multiplier*table1.loc['sway1',col]
                sway2_val = multiplier*table1.loc['sway2',col]
                if sway2_val > sway1_val:
                    better2 += 1
                else:
                    worse2 += 1
            # for sway3
            if table2.loc['sway1 to sway3',col] == '=':
                equal3+=1
            else:
                sway1_val = multiplier*table1.loc['sway1',col]
                sway3_val = multiplier*table1.loc['sway2',col]
                if sway3_val > sway1_val:
                    better3 += 1
                else:
                    worse3 += 1
        scores[dataset]['sway2'] = [equal2, better2, worse2]
        scores[dataset]['sway3'] = [equal3, better3, worse3]
    
    return scores

getDatasetScores(dataset_tables)

defaultdict(dict,
            {'SSM.csv': {'sway2': [2, 0, 0], 'sway3': [2, 0, 0]},
             'SSN.csv': {'sway2': [2, 0, 0], 'sway3': [1, 0, 1]},
             'auto2.csv': {'sway2': [4, 0, 0], 'sway3': [4, 0, 0]},
             'auto93.csv': {'sway2': [3, 0, 0], 'sway3': [3, 0, 0]},
             'china.csv': {'sway2': [1, 0, 0], 'sway3': [1, 0, 0]},
             'coc1000.csv': {'sway2': [5, 0, 0], 'sway3': [3, 2, 0]},
             'coc10000.csv': {'sway2': [1, 1, 1], 'sway3': [2, 1, 0]},
             'healthCloseIsses12mths0001-hard.csv': {'sway2': [0, 1, 2],
              'sway3': [3, 0, 0]},
             'healthCloseIsses12mths0011-easy.csv': {'sway2': [3, 0, 0],
              'sway3': [1, 1, 1]},
             'nasa93dem.csv': {'sway2': [4, 0, 0], 'sway3': [4, 0, 0]},
             'pom.csv': {'sway2': [3, 0, 0], 'sway3': [2, 0, 1]}})